In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 
import datetime 

In [2]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


     |████████████████████████████████| 204.2MB 64kB/s 
     |████████████████████████████████| 204kB 31.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=6c6b414d2732fc0853d264236f1cbd3e425b749b70804588db2253730797d556
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [7]:
from google.colab import files
uploaded = files.upload()

Saving ethereum_csv.csv to ethereum_csv.csv


In [9]:
import io

In [11]:
data = spark.read.csv("ethereum_csv.csv",header=True, inferSchema=True)

In [15]:
data.show()

+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|      date|  txVolume(USD)|adjustedTxVolume(USD)|txCount|marketcap(USD)|price(USD)|exchangeVolume(USD)|generatedCoins|         fees|activeAddresses|medianTxValue(USD)|       medianFee|averageDifficulty|paymentCount|blockSize|blockCount|
+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|2015-07-30|           null|                 null|   null|          null|      null|               null|   39311.09375|         null|           null|              null|            null| 1.21335493201E11|        null|  4449897|      6911|
|2015-07-31|           null|                 nul

In [18]:
df1 = data.dropna()

In [19]:
df1.show()

+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|      date|  txVolume(USD)|adjustedTxVolume(USD)|txCount|marketcap(USD)|price(USD)|exchangeVolume(USD)|generatedCoins|         fees|activeAddresses|medianTxValue(USD)|       medianFee|averageDifficulty|paymentCount|blockSize|blockCount|
+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|2015-08-07|  5684487.22712|        5684487.22712|   2050|           0.0|      2.83|           164329.0|   27437.65625|37.3184135113|           1086|     41.6117128722|0.00132077886624| 1.47083865066E12|        1977|  3282693|      5189|
|2015-08-08|3.78996817885E7|      3.78996817885E

In [26]:
feature_columns = data.columns[1:-1] # here we omit the final column
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [27]:
from pyspark.sql.functions import col, unix_timestamp, to_date,to_timestamp


df1 = df1.withColumn('date', to_timestamp(unix_timestamp(col('date'), 'yyyy-MM-dd').cast("timestamp")))


In [28]:
df1 = df1.withColumn('date', (unix_timestamp(col('date'), 'yyyy-MM-dd').cast("int")))

In [29]:
df1.show()

+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|      date|  txVolume(USD)|adjustedTxVolume(USD)|txCount|marketcap(USD)|price(USD)|exchangeVolume(USD)|generatedCoins|         fees|activeAddresses|medianTxValue(USD)|       medianFee|averageDifficulty|paymentCount|blockSize|blockCount|
+----------+---------------+---------------------+-------+--------------+----------+-------------------+--------------+-------------+---------------+------------------+----------------+-----------------+------------+---------+----------+
|1438905600|  5684487.22712|        5684487.22712|   2050|           0.0|      2.83|           164329.0|   27437.65625|37.3184135113|           1086|     41.6117128722|0.00132077886624| 1.47083865066E12|        1977|  3282693|      5189|
|1438992000|3.78996817885E7|      3.78996817885E

In [30]:
data_2 = assembler.transform(df1)

In [31]:
train, test = data_2.randomSplit([0.7, 0.3])

In [32]:
from pyspark.ml.regression import LinearRegression
algo = LinearRegression(featuresCol="features", labelCol="price(USD)")

In [33]:
model = algo.fit(train)

In [34]:
evaluation_summary = model.evaluate(test)

In [35]:
evaluation_summary.meanAbsoluteError

1.3145307132991313e-12

In [36]:
evaluation_summary.rootMeanSquaredError

2.046047940721807e-12

In [37]:
evaluation_summary.r2

1.0

In [38]:
predictions = model.transform(test)

In [39]:
predictions.select(predictions.columns[13:]).show() # here I am filtering out some columns just for the figure to fit

+------------+---------+----------+--------------------+-------------------+
|paymentCount|blockSize|blockCount|            features|         prediction|
+------------+---------+----------+--------------------+-------------------+
|        1973|  3508878|      5256|[3.78996817885E7,...| 2.7900000000000515|
|        2486|  3519008|      5286|[2182919.67454,21...| 1.2199999999994329|
|        3147|  3392043|      5192|[1691424.30403,16...| 1.1699999999992443|
|        3809|  3396674|      5088|[894127.676986,89...| 1.4799999999989433|
|        4025|  3411809|      5013|[697311.417317,69...| 1.3799999999989223|
|        4296|  3464583|      5097|[505362.445352,50...|  1.229999999998985|
|        4040|  3345268|      5003|[508937.45783,508...|  1.129999999998905|
|        4883|  2740989|      3577|[374923.646468,37...| 1.3499999999977117|
|        5094|  3651888|      5003|[249301.081565,24...|  1.209999999998938|
|        5247|  3740946|      4851|[252012.348897,25...|  0.940565999998791|

In [40]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [44]:
from google.colab import files
uploaded = files.upload()

Saving ethereumprice.csv to ethereumprice.csv


In [51]:
df = pd.read_csv("ethereumprice.csv")

msk = np.random.rand(len(df)) < 0.8

X = df["open"]
Y = df["high"]
X_train = np.array(X[msk]).reshape(len(X[msk]),1)
X_test = np.array(X[~msk]).reshape(len(X[~msk]),1)
Y_train = np.array(Y[msk]).reshape(len(X[msk]),1)
Y_test = np.array(Y[~msk]).reshape(len(X[~msk]),1)

In [52]:
clf = linear_model.LinearRegression()
clf.fit(X_train,Y_train)
predicted_vals = clf.predict(X_test)

ValueError: ignored